# Pytorch Deep-Learning | Model-Parallel | Docker 
***********


In diesem Notebook wird vorgeführt, wie man das Model auf GPUs aufteilt und trainiert. Dafür gibt es einige Möglichkeiten, wie man sowas realisieren kann. Wir nutzen Dask als Cluster.

Um das Model über Maschinen hinweg zu verteilten, müssen wir RPC (Remote Procedure Call) und RReF (Remote Reference) nutzen. Das kann hier und da etwas schwierig sein.

<br>
PyTorch Model-Parallel: <br>

Mit RPC können wir remote Funktionen und Aufgaben ausführen, wie z.B. dass ein Teilnetzwerk etwas berechnet. Wenn wir sagen, dass Worker:2 das Teilnetz Net2 ausführen soll, geben wir auch die Daten mit. Als Ergebnis bekommen wir eine Referenz (RReF) zu einem Objekt. Um das Ergebnis zu holen, wird es kopiert und gesendet. Die Trainingsroutine unterscheidet sich etwas von dem normalen Vorgehen. 

Die Kommunikation läuft im Hintergrund. Als Benutzer kann man dazu auch einiges einstellen. Der Benutzer entscheidet selber, wie das Netzwerk aufgeteilt wird.


Wenn alle Worker auf einer Maschine sind (Single-Node Model Parallel), reduziert sich der Aufwand der Einstellungen und Kommunikation, da PyTorch alles selber macht. Wir betrachten den Fall, dass wir das Model <u>über Nodes</u> aufteilen. 

Um es unter CPU laufen zu lassen, siehe weiter unten.


<u>Wann sollte Model-Parallel genutzt werden?</u><br>
- Das Model ist groß und passt nicht auf eine GPU.
- Das Model kann vertikal zerteilt werden.
- Mehrere GPUs sind lokal verfügbar (einfaches setup), oder über Maschinen (mehr Konfigurationsaufwand).
- Hier wird das Model vertikal aufgeteilt (in Layers oder Teilnetze) deren Output zur nächsten Node via PyTorch RPC als Input genommen wird.


Als Basis für die Verteilung und Ausführung von Funktionen auf andere Dask-Worker, stellt Saturncloud Module bereit, die für Dask-PyTorch-DDP genutzt werden. <br>
Mit Anpassungen können damit auch andere Probleme wie Model-Parallel gelöst werden. Die Konfigurationen und Verteilung laufen im Hintergrund. 


<br>

Dask und Dask.distributed sollten die gleiche Version haben.
Dask: https://www.dask.org && https://docs.dask.org/en/stable/ <br>
PyTorch: https://pytorch.org  <br>
PyTorch releases: https://github.com/pytorch/pytorch/releases <br>
Dask-PyTorch-DDP Open Source Module von Saturncloud: https://github.com/saturncloud/dask-pytorch-ddp/tree/main <br>
PyTorch RPC und RReF: https://pytorch.org/docs/stable/rpc.html <br>
DISTRIBUTED RPC FRAMEWORK: https://pytorch.org/docs/stable/rpc.html#distributed-rpc-framework <br>
PyTorch SINGLE-MACHINE MODEL PARALLEL BEST PRACTICES: https://pytorch.org/tutorials/intermediate/model_parallel_tutorial.html <br>
DISTRIBUTED PIPELINE PARALLELISM USING RPC: https://pytorch.org/tutorials/intermediate/dist_pipeline_parallel_tutorial.html#step-1-partition-resnet50-model




# 1. Aufbau und Möglichkeiten 

Es gibt verschiedene Möglichkeiten Model-Parallel zu realisieren. Wir werden uns auf ein fokussieren. Das Netzwerk wird Vertikal aufgeteilt.

Das allgemeine Trainieren kann man in 4 Bereiche aufteilen. Das Bild zeigt Bereiche, angelent an die Flynn-Notation

<img src="./pictures/rapids_flynn.PNG"  width="625px;" hight="625px;">

Wir importieren Dateien, die Klassen und Funktion erhalten, um die Benutzung zu vereinfachen.

In den Modulen selber können Einstellungen vorgenommen werden. 

In [1]:
## Tools um die Nutzung zu vereinfache
# - Das Modul selber imortiert auch pytorch_dispatcher_resulthandler
import pytorch_tools as pytorch_tools

Tools 1.0 | Setze Umgebungsvariablen
PYTORCH_DIST_BACKEND: nccl
PYTORCH_DIST_DDP_PORT: 23456
NCCL_SOCKET_NTHREADS: 4
NCCL_NSOCKS_PERTHREAD: 2
PYTORCH_MODULE_LOG: True

RPC Config:
GLOO_SOCKET_IFNAME: eno1, NCCL_SOCKET_IFNAME: eno1
TP_SOCKET_IFNAME: eno1    , START_ON_RANK: 0
PyTorch Dispatcher-/Resulthandler 1.0


Damit die Kommunikation funktioniert, muss in dem Modul `pytorch_tools.py` das Netzwerkinterface eingestellt werden. In diesem Aufbau hießt das Interface "eno1".

<img src="./pictures/ptmp_rpc.PNG"  width="365px;" hight="265px;">

Das obere Bild zeigt einen Ausschnitt des Moduls pytorch_tools.py, hier muss das richtige Interface eingestellt werden. Das Interface wird dann von allen Worker für die Kommunikation genutzt. 

`PYTORCH_MODEL_PARALLEL__START_ON_RANK` gibt an, welcher Worker nach Rang die Trainingsfunktion ausführen soll. Mehr dazu später.

In [2]:
# -- In Entwicklung -- # 
# Code für PyTorch Model-Parallel
#import pytorch_dispatcher_resulthandler

In [3]:
# Erstelle Client 
client = pytorch_tools.create_dask_client()  # Gebe IP an. Standart:  127.0.0.1:8786 (Scheduler ist da wo auch Jupyter läuft)
client

Client-IP: 127.0.0.1:8786


<Client: 'tcp://149.201.182.203:8786' processes=3 threads=3, memory=187.58 GiB>

Das Modul enhält Klassen, Funktionen und setzt Umbegungsvariablen, die auch andere Worker sehen müssen. Damit das kappt, muss das Modul im Cluster hochgeladen werden. <br> 

Wenn die Module verändert werden, müssen diese erneut hochgeladen werden. Beim Erstellen des Clients mit "pytorch_tools.create_dask_client()" wird die Funktion "scatter_files()" aufgerufen, das diese Module hochlädt.

In [4]:
## So werden die Module pytorch_tools.py und pytorch_dispatcher_resulthandler.py mit dem Cluster geteilt. 
#  *Bei Änderungen muss der Kernel in Jupyter neugestartet werden. 
#pytorch_tools.scatter_files(client) 

Wenn wir eigene Module erstellen, können wir diese auch mit dem Cluster teilen.

In [39]:
# Zeige Schlüssel. Die Ports ändern sich. 
client.has_what().keys()

dict_keys(['tcp://149.201.182.188:42179', 'tcp://149.201.182.203:44877', 'tcp://149.201.182.205:36977'])

In [14]:
# Starte alle Worker neu
client.restart_workers( client.has_what().keys() )
client.has_what().keys()

dict_keys(['tcp://149.201.182.188:37783', 'tcp://149.201.182.203:45847', 'tcp://149.201.182.205:42927'])

In [ ]:
# Startet client neu.
#client.restart()

Das Setup mit Docker kann so aussehen. Es gibt eine Node die Scheduler ist. Diese Node kann auch einen Worker haben. 

Wenn der Scheduler auch worker sein soll, muss eine zweite Konsole geöffnet werden, um den Worker zu starten. Wer am Ende Scheduler ist, ist egal, alle Knote sind gleichberechtigt.

Beispiel Setup (auf Node1 läuft auch Jupyter).: <br>
<img src="pictures/cluster_setup.PNG" width="925px;" hight="850px;"><br>
Docker logo:<br>
https://www.docker.com/company/newsroom/media-resources/![cluster_setup.png](attachment:9a543a66-02b7-4661-a8f9-a9be164d0575.png)

Das untere Bild zeigt, wie die Aufteilung von Client, Worker und Scheduler sein könnten. Nachrichten und das Model selber werden an den Client geschickt, da von dort aus die Traningsfunktion mit Dask an die Worker submitet wird. Der Resulthandler der sich auf dem Client befindet, sammelt die Nachrichten. 

<img src="./pictures/cluster_aufbau.PNG" width="725px;" hight="850px;" >

Das untere Bild zeigt 3 Worker, die jeweils mit dem Buchstaben `w` beginnen. Diese Namen werden bei der Initialisierung gesetzt und können durch die Namen im RPC-Kontext angesprochen werden. 

Dabei läuft das so ab, das w0 eine Funktion auf w1 ausführen will und als Rückgabe ein RRef bekommt. Hier vereinfach mit Frage und Antwort dargestellt.

Um beim Trainieren nicht den Output erst auf die CPU zu schieben, muss ein GPU-Map erstellt werden. Dabei wird der Hinweg und Rückweg definiert. <br>
Die Worker geben die GPU-Map in der Konsole aus, um einen Einblick zu haben. Das Erstellen wird automatisch im Hintergrund ausgeführt, jede GPU wird zu jeder anderen im Cluster gemappt.

<img src="./pictures/ptmp_rpc2.PNG"  width="765px;" hight="665px;">

<u>Unter CPU:</u><br>
Um auf CPU-Worker zu Trainieren, muss folgendes eingestellt werden:<br>
- Im Dispatcher muss die gpu-map kommentiert werden.
- Bei forward() muss .cuda() kommentiert werden (ggf .cpu() nutzen).



Als Erstes kommen die Importe, was fehlt, kann dazugenommen werden. Alle Knoten (falls Cluster) sollten dieselben Pakete installiert haben, möglichst mit derselben Version

In [11]:
# Imports die wir benötigen
###########################################################################
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


from torch.utils.data import Dataset, DataLoader
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist
import torchvision
from torch.utils.data.distributed import DistributedSampler
import torchvision.transforms as transforms

from dask.distributed import Client
import dask

import time
import sys
import os
import datetime
import pickle

import numpy as np
import torch.distributed as dist
from distributed.pubsub import Pub, Sub

## RPC/RRef für Model-Parallel
from torch.distributed.rpc import RRef 
from torch.distributed.optim import DistributedOptimizer
import torch.distributed.autograd as dist_autograd
from torch.distributed.rpc import init_rpc, rpc_async, rpc_sync, remote
import torch.distributed.rpc as rpc


# Imports für Dispatcher und Resulthandler, wird später verschwinden.
######################################

from distributed.worker import logger

from distributed.client import wait, FIRST_COMPLETED, Future
from distributed.utils import TimeoutError as DistributedTimeoutError
from time import sleep
import time
import sys
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data.distributed import DistributedSampler
import logging
import os
from typing import List, Optional
from os.path import join, exists, dirname
from distributed.utils import TimeoutError as DistributedTimeoutError
from distributed.client import wait, FIRST_COMPLETED, Future


# 2. Single-Node Model-Parallel

Das einfachste wäre, wenn lokal mehrere GPUs verfügbar sind (dann können die Layers auch mit einer Pipeline kombiniert werden). PyTorch kümmert sich dann um die Kommunikation. 

Die Layers des Models (oder Teilnetzwerke) werden mit `.to('cuda:0')` auf die GPU platziert.

Bei mehreren lokalen GPUs kann eine Pipeline erstellt werden, um das Training zu beschleinigen. Das klappt derzeit <u>nur lokal</u> auf Maschinen. <br>

Pipeline-Parallelism unter PyTorch wird in Zukunft Inter-Node fähig sein.


<br>

SINGLE-MACHINE MODEL PARALLEL BEST PRACTICES: https://pytorch.org/tutorials/intermediate/model_parallel_tutorial.html <br>
Speed Up by Pipelining Inputs: https://pytorch.org/tutorials/intermediate/model_parallel_tutorial.html#speed-up-by-pipelining-inputs <br>
PIPELINE PARALLELISM: https://pytorch.org/docs/stable/pipeline.html


# 3. Multi-Node Model-Parallel

Hier sieht das Vorgehen ganz anders aus. Hier wird MNSG hauptsächlich betrachtet. 

Das Model kann vertikal aufgeteilt werden. Wenn jede Node mehrere GPUs hat, kann man dazu zusätzlich noch PyTorch DDP nutzen, um die gesamten Trainingsdaten auf Nodes aufzuteilen und so das Trainings zu beschleunigen. Das geht nur wenn das Model auf die lokalen GPUs aufgeteilt werden kann. das Aufteilen funktioniert dann wie bei Single-Node Model-Paralell.

## 3.1. Vorgehen

Hier wird Stückweise das vorgehen erklärt. Wir befinden uns im MNSG Bereich. Das Nertwerk wird in Teilnetze geschnitten, vertikal. Jede GPU eines Workers bekommt einen Teil.



<u>Hinweis:</u><br>
Wenn beim Programmieren Fehler passieren, können die Fehlermeldungen nicht sehr aussagekräftig sein, was die Fehlersuche erschwert.<br>
Es kann vorkommen, dass nur folgendes ausgegeben wird: `terminate called without an active exception` oder `Shutdown RPC`. <br> 
Was und wo der Fehler war, wird nicht mitgeteilt.

Wenn es hängt oder bei Problemen: Worker/Notebook/Cluster, ... neustarten.<br>
Das kann manchmal helfen. Das Problem kann auftauchen, wenn Exceptions vorkommen. <br>
Es kann sein das RPC Probleme macht, wenn direkt das Training neugestartet wird, dann müssen die Worker neugestartet werden.

Die zwei unteren Abbildungen illustrieren das Prinzip. 

<p>
<img src="./pictures/ptmp_rpc3.PNG" width="565px;" hight="465px;" >
<img src="./pictures/ptmp_rpc4.PNG" width="265px;" hight="165px;" >
</p>

Es wird ein Worker ausgewählt, welcher die Trainingsfunktion ausführen soll. Die Restlichen stehen nur zur Verfügung und führen auf Anfragen, Funktionen und Klassen aus.

Das ganze Netzwerk wird in Teilnetze zerteilt. Teil 1 des Netzes kann lokal oder auf einem anderen Worker liegen. Wenn lokal- wird das Ergebnis als RRef an W1 weitergegeben. <br>
W1 macht dann an der Stelle weiter und liefert ein RRef zurück an W0. W0 gibt das RRef an W2, damit dieser sich mit der Referenz die Daten aus W1 holt. Am Ende <br> 
holt sich W0 das Endergebnis. 

Die RPC-Namen werden immer hochgezählt. Ein Worker mit Rang 3 bekommt den Namen w3.


Es gibt noch die Möglichkeit bei der `forward()` Methode die Batches zu spliten. Dafür muss am Ende ein Future von PyTorch zurückgegeben werden. <br>
Ein Beispiel der Anwendung ist unten verlinkt. Hier wird es nicht gezeigt.


<br>

Step 2: Stitch ResNet50 Model Shards Into One Module (Zeigt Batch-Splitting): https://pytorch.org/tutorials/intermediate/dist_pipeline_parallel_tutorial.html#step-2-stitch-resnet50-model-shards-into-one-module

In [5]:
####### Untere 2 Zellen werden als Module bald verfügbar sein #######
#
# Im Dispatcher unter  "if rank_int==1:  # 1" wird der Rang 1 für die Ausführung der Trainingsfunktion ausgewählt.

In [184]:
# Dask-Pytorch Dispatcher
# Open Source Project: https://github.com/saturncloud/dask-pytorch-ddp
###########################################################################

#### Das hier nutzen

import os
from typing import List, Callable, Any, Dict
from dask.distributed import Client
import torch.distributed as dist

import json


def _get_worker_info(client: Client) -> List[Dict]:

    workers = client.scheduler_info()["workers"]
    worker_keys = sorted(workers.keys())
    workers_by_host: Dict[str, List[str]] = {}
    for key in worker_keys:
        worker = workers[key]
        host = worker["host"]
        workers_by_host.setdefault(host, []).append(key)
    host = workers[worker_keys[0]]["host"]
    all_workers = []
    global_rank = 0
    for host in sorted(workers_by_host.keys()):
        local_rank = 0
        for worker in workers_by_host[host]:
            all_workers.append(
                dict(
                    worker=worker,
                    local_rank=local_rank,
                    global_rank=global_rank,
                    host=host,
                )
            )
            local_rank += 1
            global_rank += 1
    return all_workers


def run(
    client: Client,
    pytorch_function: Callable,
    *args,
    backend: str = "nccl",  # nccl | gloo
    pass_local_rank: bool = False,
    **kwargs
):
    """
    Dispatch a pytorch function over a dask cluster, and returns a list of futures
    for the resulting tasks
    """
    all_workers = _get_worker_info(client)
    world_size = len(all_workers)
    port = 23456  # pick a free port?
    host = all_workers[0]["host"]
    futures = []
    for worker in all_workers:
        if pass_local_rank:
            fut = client.submit(
                dispatch_with_ddp,
                pytorch_function=pytorch_function,
                master_addr=host,
                master_port=port,
                rank=worker["global_rank"],
                world_size=world_size,
                *args,
                local_rank=worker["local_rank"],
                backend=backend,
                workers=[worker["worker"]],
                **kwargs
            )
        else:
            fut = client.submit(
                dispatch_with_ddp,
                pytorch_function=pytorch_function,
                master_addr=host,
                master_port=port,
                rank=worker["global_rank"],
                world_size=world_size,
                *args,
                backend=backend,
                workers=[worker["worker"]],
                **kwargs
            )
        futures.append(fut)
    return futures


# pylint: disable=too-many-arguments
def dispatch_with_ddp(
    pytorch_function: Callable,
    master_addr: Any,
    master_port: Any,
    rank: Any,
    world_size: Any,
    *args,
    backend: str = "nccl",
    **kwargs
) -> Any:
    

    # These are the parameters used to initialize the process group
    master_addr = str(master_addr)
    master_port = str(master_port)
    rank_int = rank
    world_size_int = world_size
    rank = str(rank)
    world_size = str(world_size)
    
    
    os.environ["MASTER_ADDR"] = master_addr
    os.environ["MASTER_PORT"] = master_port
    os.environ["RANK"] = rank
    os.environ["WORLD_SIZE"] = world_size
    
    os.environ["GLOO_SOCKET_IFNAME"] = "eno1"  # unable to find Adress for eno4   etho zb, Netzwerkadresse
    #os.environ["NCCL_SOCKET_IFNAME"] = "eno1" 
    # Lösung: https://github.com/pytorch/tensorpipe/issues/201
    os.environ["TP_SOCKET_IFNAME"] = "eno1"
    # https://discuss.pytorch.org/t/strange-behaviour-of-gloo-tcp-transport/66651/4
    
    ## Mappe GPU:0 zu allen anderen GPUs im Cluster. 
    #  - Alle Worker beginnen mit w, gefolgt von dem Rank
    #  # Siehe https://pytorch.org/docs/stable/rpc.html#torch.distributed.rpc.TensorPipeRpcBackendOptions.set_device_map
    # -- Für CPU:
    #   - Kommentiere gpu-map unten bei der init_method. 
    gpu_map = {}
    for i in range(world_size_int):
        gpu_map[f'w{i}'] = {0: 0}
    del gpu_map[f'w{rank_int}']    # Lösche eigenen Eintrag 
    print(f"Rank: {rank_int} gpu_map: {gpu_map}")
    ## RPC Backend Optionen
    options=rpc.TensorPipeRpcBackendOptions(
        num_worker_threads=18, #128
        rpc_timeout=60,
        init_method=f'tcp://{master_addr}:{master_port}', device_maps=gpu_map  )
    
   
    
    available_rpc_worker = []
    for i in range(world_size_int):
             available_rpc_worker.append(f"w{i}")
   
    val=100
    try: 
        
        if rank_int == 1:  # 1
        
            rpc.init_rpc(f"w{rank_int}", rank=rank_int, world_size=world_size_int, rpc_backend_options=options, backend=rpc.BackendType.TENSORPIPE)            
            print(f"Init master, rank: {rank_int}") 
            
            json_dumps_str= json.dumps(available_rpc_worker)
            os.environ["RPC_WORKER_LIST"] = json_dumps_str
            os.environ["RPC_WORKER_NAME"] = f"w{rank_int}"
            val = pytorch_function(*args, **kwargs)
     
        else:
            
            print(f"init worker, rank: {rank_int}, RPC name: w{rank_int}")
            os.environ["RPC_WORKER_NAME"] = f"w{rank_int}"
            rpc.init_rpc(f"w{rank_int}", rank=rank_int, world_size=world_size_int, rpc_backend_options=options, backend=rpc.BackendType.TENSORPIPE) 
                        
    finally:
        rpc.shutdown()
        print("Shutdown RPC")
        return val

In [185]:
from distributed.pubsub import Pub, Sub
from typing import List, Optional, Dict
from distributed.client import wait, FIRST_COMPLETED, Future
from os.path import join, exists, dirname
from distributed.utils import TimeoutError as DistributedTimeoutError
import logging
import os

from torch.distributed.rpc import init_rpc, rpc_async, rpc_sync, remote
import torch.distributed.rpc as rpc


# Dask-Pytorch Dispatcher
# Open Source Project: https://github.com/saturncloud/dask-pytorch-ddp
###################################################################################################


from typing import List, Callable, Any, Dict

class DaskResultHandler:
    """
    This class use Dask pubsub infra to pass intermediate results back from PyTorch
    jobs to the client.
    """

    def __init__(self, pub_sub_key:str="my_channel", trainingpath:str="template_pytorch/"):
        """Init Class
        Hier kann man zu Beginn Ordner oder Pfade festlegen.
        Ein Pfad, den man übergeben will, sollte so aussehen: "training/ordner1/ordner2/"
        
        Wenn nötig, können auch Funktionen ausgeführt werden, um Pfade, etc. zu erstellen. 
        
        trainingpath: Wo das Model gespeichert wird. 
        pub_sub_key:  Publisher/Subscriber Channel Name.
        _setup_working_dir: Erstellt den Ordner "training", wenn nicht vorhanden
        """
        
        self.training_path = trainingpath
        self.pub_sub_key   = pub_sub_key
        #self._setup_working_dir(trainingpath)
        
        # Erstelle Pfade, sonstige Vorbereitungen... 
        # _setup_working_dir()

        
    @classmethod
    def _get_all(cls, sub: Sub):
        """Auslesen des Channels:
        Geben die Daten zurück die jede Epoche von einem Worker hochgeladen werden. 
        - Host ist meist Subscriber und hört allen Topics zu.
        - Es kann mehrere Nachrichten und Channels geben.
        """
        while True:
            try:
                yield sub.get(timeout=1.0)
            except DistributedTimeoutError:
                break
                

    def _get_results(self, futures: List[Future], raise_errors: bool = True):
        """Get Dask results.
        Hier erstellen wir ein Subscriber sub_stats, der die Daten aus dem Channel pub_sub_key auslesen soll. 
        
        """    
        sub_stats = Sub(self.pub_sub_key)  

        
        while True: 
            
            # Für Subscriber, get Channel data.
            for obj in self._get_all(sub_stats):     
                yield obj
            
            # keine Futures? => break. 
            if not futures:
                break
                
            try:
                # Dask:   wait(fs[, timeout, return_when])      Wait until all/any futures are finished
                # Read here: https://distributed.dask.org/en/stable/api.html
                result = wait(futures, 5, FIRST_COMPLETED)  #0.1
            except DistributedTimeoutError:
                continue

            for fut in result.done:     
                try:                   
                    fut.result()  
                    
                except Exception as e:  # pylint: disable=broad-except
                    logging.exception(e)
                    
                    if raise_errors:
                        raise
                        
            futures = result.not_done


    def process_results(self, futures: List[Future], raise_errors: bool = True) -> None:
        
        """Verarbeitung der Daten der Futures die von Dask geliefert werden.
        Die Ergebnisse kommen als Liste an.
        
        Die Liste "futures" enthält alles, was wir mit dem Publisher hochladen.
        Das kann das Model sein (als Dict) und weitere Werte wie Loss, Acc, ...
        
        Hier kann eine Bedingung eingefügt werden, um das Training zu stoppen und das Model zu speichern. 
        
        Mit torch.save wird das Model gespeichert. Oder implementiere eine eigene Methode für das Speichern. 
        """

        for result in self._get_results(futures, raise_errors=raise_errors):
            
            
            
            if "msg" in result:
                msg = result['msg']
                print(msg)
            
            if "model_dict" in result:
                model_dict_data = result['model_dict']
                
                print("saving...")
                
                path   = model_dict_data['path']
                kwargs = model_dict_data['kwargs']
                
                
                if len(kwargs.keys()) !=0 :   # Checkpoint
                    torch.save({
                        'model_state_dict': model_dict_data['model_state_dict'],
                        'kwargs':           kwargs
                         }, str(path+".ckpt"))
                        
                else:                         # .pt oder .pth 
                    print(f"Saving Model to {path}")
                    torch.save(model_dict_data['model_state_dict'], str(path+".pt")) 

## 3.2 Das Netz aufteilen

Jetzt wird ein Netzwerk vertikal aufgeteilt. Als Beispiel nehmen wir AlexNet, das Bilder klassifizieren soll. <br>
Mit 3 Nodes (eine GPU per Node) kann das Model in 3 Stücke zerteilt werden. 

Wie die Aufteilung gemacht wird, kann die Trainingszeit beeinflussen. 

Teilnetzwerke können die Layers in eine Pipeline stecken.

So sieht das ganze Netzwerk aus:

In [14]:
# AlexNet: https://blog.paperspace.com/alexnet-pytorch/

# - Klassen: 10 
# Unser AlexNet.:
class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))  # Output geht zu Teil 2
        #----------------------------------------------------------------------------- Schnitt, Netz Teil 1
        self.layer3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer5 = nn.Sequential(
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2)) # Output geht zu Teil 3
        #----------------------------------------------------------------------------- Schnitt, Netz Teil 2
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(9216, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(4096, num_classes))
        #----------------------------------------------------------------------------- Rest: Netz Teil 3
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        #-------------------------
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = out.reshape(out.size(0), -1)
        #-------------------------
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out
        #-------------------------

Damit PyTorch RPC funktioniert, müssen die Teilnetzwerke in einer Datei stehen, die jeder Worker dann bekommt.

<img src="./pictures/ptmp_rpc5.PNG" width="765px;" hight="565px;" >

In Jupyter kann mit der Magic `%%writefile pytorch_AlexNetMP.py` eine Datei geschrieben werden.

In dieser Datei wird das Netzwerk aufgeteilt. Die Details werden unten besprochen.

In [186]:
%%writefile pytorch_AlexNetMP.py


""" Hilfsmethoden """
#### -------------------------------------------- ####
def _call_method(method, rref, *args, **kwargs):
    return method(rref.local_value(), *args, **kwargs)  #rref.local_value()

def _remote_method(method, rref, *args, **kwargs):
    return rpc.rpc_sync(
        rref.owner(),
        _call_method,
        args=[method, rref] + list(args),
        kwargs=kwargs
    )

def _parameter_rrefs(module):
    param_rrefs = []
    for param in module.parameters():
        param_rrefs.append(RRef(param))
    return param_rrefs
#### -------------------------------------------- ####



## Importiere alles was für die Netzwerke und etc. gebraucht wird. 
#  PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributed as dist
#  Für PyTorch RPC 
from torch.distributed.rpc import RRef, init_rpc, rpc_async, rpc_sync, remote
import torch.distributed.rpc as rpc
import threading

##################################### Teile auf. #####################################


## -- Teil 1 -- ##
# Teil 1 soll lokal sein.
#   ** Kann Remote zu sich selber sein.

class Net1(nn.Module):
    def __init__(self, num_classes=10):
        super(Net1, self).__init__()
        
        self.device = torch.device(0)  # Wähle Device  GPP / CPU
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2)).to(self.device)   # Layers werden Explizit auf GPU gepackt, bei CPU gff. nicht erforderlich.
        self.layer2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2)).to(self.device)
        
        # Das wird hinzugefügt.
        self._lock = threading.Lock()     
        
        
        self.savename="net1"   # Name des Netzes das beim Speichern gesetzt wird. 
         
    def forward(self, x):
        with self._lock:
            out = self.layer1(x)
            out = self.layer2(out)
        return out.cuda()     # Durch die GPU-Map müssen die Daten nicht vorher auf die CPU geschoben werden, das spart zeit.
                              # - Ohne GPU:  nur out oder out.cpu()
    
    # Parameter werden als RRef zurückgegeben.
    def parameter_rref(self):
        return [RRef(p) for p in self.parameters()]
    
    # Speichern und Laden des Models #
    def save_local(self, path="./"):
        print(f"net1 save: {str(path+self.savename+'.pt')}")
        torch.save(self.state_dict(), str(path+self.savename+".pt") )
    
    def load_local(self, path="./"):
        print(f"net1 load from {str(path+self.savename+'.pt')}")
        self.load_state_dict(torch.load(str(path+self.savename+".pt")))
    


## -- Teil 2 -- ##
# Teil 2 ist Remote und nicht lokal wie Teil 1.

class Net2(nn.Module):
    def __init__(self, num_classes=10):
        super(Net2, self).__init__()
        
        self.device = torch.device(0)
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU()).to(self.device)
        self.layer4 = nn.Sequential(
            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU()).to(self.device)
        self.layer5 = nn.Sequential(
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2)).to(self.device)
        
        # Das wird hinzugefügt.
        self._lock = threading.Lock() 
        
        self.savename="net2"
      
    
    def forward(self, x_rref): # RRef #  
        x = x_rref.to_here().to(self.device)   # Neu: Netz 2 bekommt RRef vom Ergebnis, holt sich die Kopie vom Besitzer des RRefs 
        with self._lock:       
            out = self.layer3(x)
            out = self.layer4(out)
            out = self.layer5(out)
            out = out.reshape(out.size(0), -1)
        return out.cuda()  

    
    # Parameter werden als RRef zurückgegeben.
    def parameter_rref(self):
        return [RRef(p) for p in self.parameters()]
    
    
    def get_model_state_dict(self):
        return self.state_dict()
    
        
    def save_local(self, path="./"):
        print(f"net2 save: {str(path+self.savename+'.pt')}")
        torch.save(self.state_dict(), str(path+self.savename+".pt") )
    
    def load_local(self, path="./"):
        print(f"net2 load from {str(path+self.savename+'.pt')}")
        self.load_state_dict(torch.load(str(path+self.savename+".pt")))
    
    
    
## -- Teil 3 -- ##

# - Selbes vorgehen. Bei anderen Netzen genau so. 
class Net3(nn.Module):
    def __init__(self, num_classes=10):
        super(Net3, self).__init__()
        
        self.device = torch.device(0)
       
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(9216, 4096),
            nn.ReLU()).to(self.device)
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU()).to(self.device)
        self.fc2= nn.Sequential(
            nn.Linear(4096, num_classes)).to(self.device)
        
        # Das wird hinzugefügt.
        self._lock = threading.Lock()
        
        self.savename="net3"
            
        
    def forward(self, x_rref): # RRef #
        x = x_rref.to_here().to(self.device) # Neu: Netz 3 bekommt RRef vom Ergebnis, holt sich die Kopie vom Besitzer des RRefs 
        with self._lock:
            out = self.fc(x)
            out = self.fc1(out)
            out = self.fc2(out)
                
        return out.cuda()
    
    # Parameter werden als RRef zurückgegeben.
    def parameter_rref(self):
        return [RRef(p) for p in self.parameters()]
    
    def save_local(self, path="./"):
        print(f"net3 save: {str(path+self.savename+'.pt')}")
        torch.save(self.state_dict(), str(path+self.savename+".pt") )
    
    def load_local(self, path="./"):
        print(f"net3 load from {str(path+self.savename+'.pt')}")
        self.load_state_dict(torch.load(str(path+self.savename+".pt")))
    
    

    
## --- Führe alles zusammen. --- ##
#
class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.device = torch.device(0)

        # Lokal
        self.part1 = Net1().to(self.device)  # Lokal     w1
        # Node w1 führt Trainingsfunktion aus
        self.part2 = rpc.remote("w0", Net2)  # Remote    w0
        self.part3 = rpc.remote("w2", Net3)  # Remote    w2
        
    def forward(self, x):
         
        x_rref1 = RRef(self.part1.forward(x))                 # Ergebnis zu RRef, damit die Referenz an Part2(Netz2) gesendet werden kann.
        x_rref2 = self.part2.remote().forward( x_rref1 )      # RRef des Ergebnisses wird zurück gegeben. Diese soll weitergereicht werden.
        # Bei weiteren:  x_rref n = self.part n.remote().forward( x_rref n-1 ), Es sollte immer eine Referenz zurückgegeben werden.
        data1   = self.part3.rpc_sync().forward( x_rref2 )    # data1 ist ein Tensor. (Für das Batch-Splitting muss es ein Future von PyTorch sein)
        return data1  # ist Tensor
        
    # Methode erweitert Liste um Parameter. 
    def parameter_rrefs(self):
        remote_params = []
        # create RRefs for local parameters
        remote_params.extend(_parameter_rrefs(self.part1))
        # get RRefs remote
        remote_params.extend(self.part2.remote().parameter_rref().to_here())
        remote_params.extend(self.part3.remote().parameter_rref().to_here())

        return remote_params
    
    
    """Speichern und Laden der Modelle.
    - Die Teilmodelle werden dort gespeichert, wo sie ausgeführt werden.
      So werden diese auch geladen.
    """
        
    def saveModelShardsLokalOnWorker(self, path:str="./"):
        self.part1.save_local(path)
        
        self.part2.remote().save_local(path)
        self.part3.remote().save_local(path)
        
    def loadModelShardsLokalOnWorker(self, path:str="./"):
        self.part1.load_local(path)
        
        self.part2.remote().load_local(path)
        self.part3.remote().load_local(path)
        
        
        
    
        
######### Datei: writefile pytorch_AlexNetMP.py

Overwriting pytorch_AlexNetMP.py


Dann muss die Datei mit dem Cluster geteilt werden.


In [187]:
client.upload_file('pytorch_AlexNetMP.py')
import pytorch_AlexNetMP

<u>Hinweis zum Dataloader:</u><br>
Um die Parameter num_workers und prefetch_factor zu nutzen, muss unter Dask eingestellt werden, dass die Worker nicht als Daemon Prozesse starten. Diese Parameter können die Trainingszeit verkürzen. <br>
`num_workers:int` Erstelle n-Prozesse zum laden der Daten. <br>
`prefetch_factor:int` Lade n-Batches vor. 

Damit das funktioniert, muss <u>vor</u> dem start der Worker eine Umgebungsvariable gesetzt werden. Bei jedem Worker muss es gesetzt sein. <br>
Setze: `DASK_DISTRIBUTED__WORKER__DAEMON=False`. Der Standartwert ist `True`.


Eine Abfrage kann so gemacht werden:<br>
`dask.config.get("distributed.worker.daemon")`. 


<br>


Setzen und abfragen von Einstellungen in Dask: https://docs.dask.org/en/latest/configuration.html#conversion-utility


Um eine andere Sicht auf die Funktionsweise zu bekommen, wird eine kleine Funktion im RPC-Kontext ausgeführt.  <br>
Um auszuwählen welcher Worker die Funktion ausführen soll: Im Dispatcher => `if rank_int==1:  # 1`. 

In [151]:
%%writefile my_file.py
import time

# Klassen und Funktionen... 
def add(a,b):
    return a+b

def mul(a,b):
    return a*b

def dev(a,b):
    time.sleep(10)
    return a/b

Overwriting my_file.py


In [163]:
client.upload_file('my_file.py')
import my_file

Beachte, wenn hier ein Fehler auftaucht, kann es sein, dass es keine klare Fehlermeldung gibt. <br>
Z.B. wenn `print()` groß geschrieben wird: `Print()`.

In [162]:
# Kontext: RPC und RRef
def my_rpc_function():
    pub_stats  = Pub("my_channel")                     
    rpc_worker_name = os.getenv('RPC_WORKER_NAME') 
    rpc_worker_list = os.getenv('RPC_WORKER_LIST')
    
    msg = f"Hallo, I'm the RPC-Worker {rpc_worker_name}"
    print(msg)                  # Konsole
    pub_stats.put({'msg': msg}) # Jupyter 
    
    msg = f"RPC-Worker List: {rpc_worker_list}"
    pub_stats.put({'msg': msg}) # Jupyter 
    
    rref_1 = rpc.remote("w2", my_file.add, args=(3, 3,)) # Nur ein "Pointer"
    msg = f"\nrref_1 ist: {rref_1} \nHole Ergebnis, jetzt: {rref_1.to_here()}"
    del rref_1
    pub_stats.put({'msg': msg}) 
    

    time.sleep(1)
    

In [164]:
# Starte Training...
# - Wird später als Modul verpackt. - #
workers = client.has_what().keys()
n_workers = len(workers)
print(f"Worker: {n_workers}")
client.wait_for_workers(n_workers)
rh = DaskResultHandler()
time.sleep(1)
futures = run(client, my_rpc_function)
%time rh.process_results(futures, raise_errors=False)
time.sleep(1)
del rh, futures

Worker: 3
Hallo, I'm the RPC-Worker w1
RPC-Worker List: ["w0", "w1", "w2"]

rref_1 ist: UserRRef(RRefId = GloballyUniqueId(created_on=1, local_id=0), ForkId = GloballyUniqueId(created_on=1, local_id=1)) 
Hole Ergebnis, jetzt: 6
CPU times: user 22.5 ms, sys: 0 ns, total: 22.5 ms
Wall time: 4.92 s


In [165]:
#client.restart()

Jetzt zum Training

In [188]:
## Funktion im RPC-Kontext ##

def train():
    pub_stats  = Pub("my_channel")            # Channel          
    rpc_worker_name = os.getenv('RPC_WORKER_NAME') # RPC Name der bei der Initialisierung gesetzt wird.
    device = torch.device("cuda")  # 'cuda': Nutze alle GPUs | 0: nutze GPU 0 | 'cpu': Nutze CPU, Backend auf gloo stellen | "cuda" if torch.cuda.is_available() else "cpu")
    
    """Edits: 
    num_epochs: Epochen
    batch_size: Batchgröße 
      - Die Auswahl von batch_size kann die Trainingszeit und Genauigkeit beeinflussen.
    transform: Transformiere Bilddaten
    trainset: Erstellt ein PyTorch Dataset. 
      - Wie man eigene Datasets erstellt findet, man weiter unten.  
    """
    
    num_epochs = 5 # 5
    batch_size = 128   
    """ """
    transform = transforms.Compose([
        transforms.Resize((227,227)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.4914, 0.4822, 0.4465],
            std=[0.2023, 0.1994, 0.2010])
        ])
    
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True,  download=True, transform=transform)
    # Beim Dataloader kann num_workers und prefetch_factor übergeben werden. In Dask bevor die Worker starten: export DASK_DISTRIBUTED__WORKER__DAEMON=False
    loader = DataLoader(trainset, batch_size=batch_size, num_workers=2, prefetch_factor=2, pin_memory=True)  # num_workers=4, prefetch_factor=2,  pin_memory=True
    
    model = pytorch_AlexNetMP.AlexNet().to(device)   # Erstelle Model und setze auf GPU  
    
    criterion = nn.CrossEntropyLoss()
    
    # Der DistributedOptimizer bekommt die Modelparameter als RRefs. 
    optimizer = DistributedOptimizer(
        optim.SGD,               # Optimierer Klasse
        model.parameter_rrefs(), # Parameter
        lr=0.005,                # Rest sind **kwargs für optim.SGD
        momentum=0.9,
        weight_decay = 0.005    
    )
    
    
###########################################################################
    for epoch in range(num_epochs):

        correct = 0
        for i, (batch_x, batch_y) in enumerate(loader):
            
            batch_x = batch_x.to(device) 
            batch_y = batch_y.to(device)
            
            with dist_autograd.context() as context_id:   # Neu dazugekommen 
                
                print(i)  # Um zu sehen, wie schnell die Batches durchgehen. 
                   
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y) 

                dist_autograd.backward(context_id, [loss])  # Übergebe dist_autograd ID und loss.
                optimizer.step(context_id)
                 
                ########## Evaluierung nach dem PyTorch Basic Tutorial
                _, predicted = torch.max(outputs.data, 1)
            
                if i % 100==0:
                    print("Batch ", i)
          
                if torch.cuda.is_available():
                         correct += (predicted.cpu() == torch.flatten(batch_y) .cpu()).sum()
                else:
                        orrect += (predicted == torch.flatten(batch_y) ).sum()
                #########
        
        model.saveModelShardsLokalOnWorker()  # Speichere Model 
        accuracy = 100 * correct / len(trainset)
           
        # msg: Schreibe Nachricht die in Jupyter Ausgaben generiert.
        msg=f"Epoche: ({epoch+1}/{num_epochs}), loss: {loss.item()}, acc: {accuracy}"
        pub_stats.put({'msg': msg})

<b>*</b> Um das Training erneut durchzuführen, müssen die Worker ggf. neugestartet und die  .py Datei hochgeladen werden.

In [189]:
# Dask 
workers = client.has_what().keys()
n_workers = len(workers)
print(f"Worker: {n_workers}")

client.wait_for_workers(n_workers)

rh = DaskResultHandler()

Worker: 3


In [190]:
futures = run(client, train)
%time rh.process_results(futures, raise_errors=False)
time.sleep(1)
del rh, futures

Epoche: (1/5), loss: 1.1037698984146118, acc: 48.48400115966797
Epoche: (2/5), loss: 0.9124933481216431, acc: 66.1500015258789
Epoche: (3/5), loss: 0.7793256044387817, acc: 72.61599731445312
Epoche: (4/5), loss: 0.6951331496238708, acc: 76.2699966430664
Epoche: (5/5), loss: 0.5705133080482483, acc: 78.80400085449219
CPU times: user 550 ms, sys: 85.6 ms, total: 636 ms
Wall time: 18min 29s


2023-11-19 11:29:10,096 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
ERROR:distributed.client:Failed to reconnect to scheduler after 30.00 seconds, closing client


Diese Methoden zeigt, wie Aufteilung der Worker ist. Das wird für die Verteilung genutzt. <br>
`pytorch_tools.listDaskWorker(client)` <br>


global_rank 0 => RPC-Worker Name w0

In [175]:
# Aufteilung der Worker
pytorch_tools.listDaskWorker(client)

[{'worker': 'tcp://149.201.182.188:46051',
  'local_rank': 0,
  'global_rank': 0,
  'host': '149.201.182.188'},
 {'worker': 'tcp://149.201.182.203:40883',
  'local_rank': 0,
  'global_rank': 1,
  'host': '149.201.182.203'},
 {'worker': 'tcp://149.201.182.205:36163',
  'local_rank': 0,
  'global_rank': 2,
  'host': '149.201.182.205'}]

## 3.3 Evaluieren in der Trainingsloop

Bei dem Trainieren kann man neben dem normalen Vorgehen auch das Model gleichzeitig evaluieren, dazu muss man auch einiges beachten. Eines der wichtigen Dinge ist, dass bestimmte Teile auf der GPU und andere auf der CPU sein müssen, damit das Evaluieren gut geht, sonst tauchen Fehler auf.

Das folgende Beispiele zeigt eine Trainingsschleife, wo auch die Genauigkeit getestet wird. Aufkommende Fehlern weisen auf die Bereiche hin, es kann vorkommen das manche Fehlermeldungen nicht sehr aussagekräftig sind, meist findet man auf GitHub und andere Webseiten Lösungen dazu.

##  4. Details zu Datasets und Dataloader

Das PyTorch Dataset ermöglicht das einfache Umgehen  mit den Daten. Die Funktion braucht 3 Methoden: `__init__`, `__len__` und `__getitem__`. In der init Methode kann man z.B. die Daten nochmal verändern oder von dort aus laden. 

Ohne ein Dataset können Daten so direkt geladen werden: `Train_loader = DataLoader(torch.utils.data.TensorDataset(X_train, y_train), shuffle=True, batch_size=8)`.<br>
Es ist leicht und schnell erledigt. Man kann auch ein eigenes PyTorch Dataset erstellen. Beispiele dazu folgen.

Der Dataloader wird beim Training genutzt und hat Zugriff auf das Dataset.

Hier bei Model-Parallel, wird nur ein Worker die Daten für das Training liefern. 


<br>

PyTorch Dataset und Dataloader: https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#loading-a-dataset  <br>
PyTorch Dataloader https://pytorch.org/docs/stable/data.html#data-loading-order-and-sampler

Das Laden und Verarbeiten der Daten kann als Funktion ausgelagert werden. <br>
Ggf. müssen diese Funktionen auch als Datei hochgeladen werden.

<u>Hinweis:</u><br>
Wenn es zu einem Pickelerror kommt, müssen bestimmte Klassen und Funktionen in eine .py Datei geschrieben und von jedem Worker importiert werde. <br>
Dateien können mit `client.upload_file('myfile.py')` hochgeladen werden.

## 4.1 Eigenes PyTorch Dataset

Das Beispiel zeigt, wie man derzeit Bilddaten aus dem HDFs laden könnte (so ähnlich auch für lokale Daten). 

Als erstes listen wir alle Pfade auf, danach laden wir die Bilder. 

In [ ]:
# Liste alle Pfade der Bilder auf.
def get_filenames(fs): 
    
    classes =['dog', 'chicken']   # Klassen dir wir haben (und auch verzeichnisse) 
    data = []
    file_location = []
    # HDFs verbindung
    hdfs = fs.HadoopFileSystem("hdfs://sun.bigdata.fh-aachen.de", port=9000, user="schechtel")
        
    for i in classes:
        files = hdfs.get_file_info(fs.FileSelector(f'/project/schechtel/animals/{i}')) # Pro Verzechniss werden alle Pfade aufgelistet. 
        print(f"Class: {i} \t items: {len(files)}")
        for path in files:
            data.append([path.path, i])  # Data: [...PNG , Klasse Dog]
            
    np.random.shuffle(data) # Shuffle

    return data

In [ ]:
import skimage

# Dataset
class create_trainset(Dataset):
    def __init__(self, data_paths):  
        """
        data_paths: Die Pfade die mit get_filenames ermittelt wurde.
        - Könnte auch hier gemacht werden... 
        """
        self.data_paths = data_paths  # Data: [...PNG , Klasse Dog]
        self.hdfs = fs.HadoopFileSystem("hdfs://sun.bigdata.fh-aachen.de", port=9000, user="schechtel")  # HDFS  Verbindung 
        # Sonstige Angaben... 
        self.img_dim    = (32, 32) # 227
        self.transform = transforms.Compose(
                           [transforms.ToTensor(),
                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))] )
        self.class_map = {'dog':0, 'chicken': 1}
          
    def __len__(self):
        return len(self.data_paths)  # Länge des Datasets 

    def __getitem__(self, idx):      # Iteriere mit idx:   __getitem__(idx)
         
        img_path, class_name = self.data_paths[idx]    # img_path=...PNG,  class_name=... dog

        loded_image = skimage.io.imread( self.hdfs.open_input_file(img_path) )
        img = Image.fromarray(np.uint8(loded_image)) 
        img = fn.center_crop(img, self.img_dim) 
        img = self.transform(img) 
        
        class_id = self.class_map[class_name]   # dog => 0
        
        img_tensor = torch.from_numpy(np.asarray(img).copy())
        
        class_id = torch.tensor([class_id]) 

        return img_tensor, class_id 
        

Wenn die Daten lokal auf dem Rechner sind, könnte das Laden der Daten so aussehen wie unten. <br>
Im Dataset ermitteln wir mittels Python alle Bildnamen und Pfade.

Im Dataset können auch Subsets von Pfaden für veschiedene Worker erstellt werden, oder auch Datasets für das Testen. 

In [ ]:
class mask_dataset(Dataset):
    def __init__(self):
        self.classes=['with_mask','without_mask']             # 2 Klassen
        self.class_map={'with_mask':0, 'without_mask':1 }
        self.dir="01work/Datasets/facemask/archive/"          # ../archive/with_mask  |  ../archive/without_mask  
        self.img_paths=[]
        
        for klass in self.classes:
            tmp_list = os.listdir(f"{self.dir}{klass}")   # os.listdir(): Liste Verzeichniss auf und gebe als Liste zurück 
            print(f"Klasse: {klass}, Items: {len(tmp_list)}")
            for path in tmp_list:
                self.img_paths.append([f"{self.dir}{klass}/{path}", self.class_map[klass]]) #   [....PNG , with_mask => 0 ]
        print(f"Gesamte Items: {len(self.img_paths)}")    
        
        # Shuffle...
        # worker n soll diese Pfade Pfade bekommen... 
        # Testimg = ...
                
    def __len__(self):
          return len( self.img_paths)

    def __getitem__(self, idx):
        
        img = skimage.io.imread( self.img_paths[idx][0] )
        img = Image.fromarray(np.uint8(img))
        img_tensor = torch.from_numpy(np.asarray(img).copy())
        
        return img_tensor, torch.tensor(int(self.img_paths[idx][1]))

# 5. Model Speichern

Bei dem momentanen Stand wird das Model lokal gespeichert, ohne die Modelle selber mit dem Client zu teilen. 

Wenn das Model geladen werden soll, werden die einzelnen Teilmodelle im RPC-Kontext geladen und stehen zur Verfügung. 

Wie das Speichern und Laden realisiert wird, kann vom Anwender programmiert werden. Standardmäßig wird das Model lokal als Dict gespeichert. 

<br>

SAVING AND LOADING MODELS: https://pytorch.org/tutorials/beginner/saving_loading_models.html#saving-and-loading-models

<img src="./pictures/ptmp_rpc6.PNG" width="865px;" hight="665px;" >

# 6. Model Laden und Evaluieren (GPU)

Um das Model wieder zu laden und zu Nutzen, muss eine Funktion verwendet werden, die im RPC-Kontext läuft. 

In [176]:
# Lade Datei hoch
client.upload_file('pytorch_AlexNetMP.py')
import pytorch_AlexNetMP

In [177]:
## Funktion im RPC-Kontext ##

def eval_func():
    pub_stats  = Pub("my_channel")            # Channel          
    device = torch.device("cuda")  # 'cuda': Nutze alle GPUs | 0: nutze GPU 0 | 'cpu': Nutze CPU, Backend auf gloo stellen | "cuda" if torch.cuda.is_available() else "cpu")
   

    transform = transforms.Compose([
        transforms.Resize((227,227)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.4914, 0.4822, 0.4465],
            std=[0.2023, 0.1994, 0.2010])
        ])
    
    # Cifar10 Testset
    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    # Dataloader                                     
    testloader = torch.utils.data.DataLoader(testset, batch_size=4,shuffle=False)

    correct = 0
    total = 0
    
    model = pytorch_AlexNetMP.AlexNet().to(device) # Erstelle AlexNet Model aus Teilnetzen. 
    model.loadModelShardsLokalOnWorker()           # Die Teilnetze sollen das gespeicherte Laden.
    # Jetzt ist das Netz wieder verfügbar # 

   
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in testloader:
            images, labels = data
        
            images = images.cuda()      # CUDA for GPU usage  
            labels = labels.cuda()      # CUDA for GPU usage 

            # calculate outputs by running images through the network
            outputs = model(images)  
            
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    msg=f'Accuracy of the network on the 10000 test images: {100 * correct // total} %'

    pub_stats.put({'msg': msg})

In [178]:
# Dask 
workers = client.has_what().keys()
n_workers = len(workers)
print(f"Worker: {n_workers}")

client.wait_for_workers(n_workers)

rh = DaskResultHandler()

Worker: 3


In [179]:
futures = run(client, eval_func)
%time rh.process_results(futures, raise_errors=False)
time.sleep(1)
del rh, futures

Accuracy of the network on the 10000 test images: 70 %
CPU times: user 32.3 ms, sys: 4.8 ms, total: 37.1 ms
Wall time: 47.1 s


## 6.2. Weiteres

Die ganze Evaluierung kann in Funktionen ausgelagert und vereinfacht werden.<br>
Mit Torchmetric kann die Evaluierung vereinfacht werden und mehr. 

Um mehr Übersicht zu haben, kann auch das TensorBoard von TensorFlow für PyTorch eingesetzt werden.

<br>

WELCOME TO TORCHMETRICS: https://torchmetrics.readthedocs.io/en/stable/ <br>
TORCHMETRICS Beispiel: https://lightning.ai/docs/pytorch/stable/ecosystem/metrics.html <br>
TORCH.UTILS.TENSORBOARD: https://pytorch.org/docs/stable/tensorboard.html

In [180]:
!pip install torchmetrics  # Muss überall installiert sein 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 30.6 MB/s eta 0:00:00


In [181]:
import torchmetrics
def eval_func():
    pub_stats  = Pub("my_channel")            # Channel          
    device = torch.device("cuda")  # 'cuda': Nutze alle GPUs | 0: nutze GPU 0 | 'cpu': Nutze CPU, Backend auf gloo stellen | "cuda" if torch.cuda.is_available() else "cpu")
   
    metric = torchmetrics.Accuracy(task="multiclass", num_classes=10).to(device)
    
    transform = transforms.Compose([
        transforms.Resize((227,227)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.4914, 0.4822, 0.4465],
            std=[0.2023, 0.1994, 0.2010])
        ])
    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)                                   
    testloader = torch.utils.data.DataLoader(testset, batch_size=4,shuffle=False)

    model = pytorch_AlexNetMP.AlexNet().to(device) 
    model.loadModelShardsLokalOnWorker()           
   
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in testloader:
            images, labels = data
        
            images = images.cuda()      # CUDA for GPU usage  
            labels = labels.cuda()      # CUDA for GPU usage 
            outputs = model(images) 
            
            acc = metric(outputs, labels)
         
    acc = metric.compute()        
    msg=acc
    pub_stats.put({'msg': msg})
    metric.reset()


In [ ]:
client.shutdown()